In [1]:
%load_ext autoreload
%autoreload 2
#!pip install sb3-contrib

In [12]:
from sb3_contrib import MaskablePPO
from stable_baselines3.common.callbacks import BaseCallback
import time

In [13]:
from apad_env import APADEnv

In [14]:
class TimerCallback(BaseCallback):
    def __init__(self):
        super().__init__()
        self.start_time = time.time()
    
    def _on_step(self):
        if self.num_timesteps % 1000 == 0:
            elapsed = time.time() - self.start_time
            rate = self.num_timesteps / elapsed
            remaining = (self.locals['total_timesteps'] - self.num_timesteps) / rate
            print(f"Step {self.num_timesteps}, {elapsed:.0f}s elapsed, {remaining:.0f}s remaining")
        return True

In [15]:
env = APADEnv()

In [16]:
env.reset()
model = None
model = MaskablePPO(
    "MlpPolicy",
    env,
    learning_rate=1e-5,
    tensorboard_log="./maskable_ppo_logs_4/",
    verbose=1,
    #ent_coef=0.1
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [17]:
total_timesteps = 200000
checkpoint_interval = 50000
for i in range(0, total_timesteps, checkpoint_interval):
    remaining_steps = min(checkpoint_interval, total_timesteps - i)
    if i == 0:
        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=True, callback=TimerCallback())
    else:
        model.learn(total_timesteps=remaining_steps, reset_num_timesteps=False, callback=TimerCallback())
    model.save(f"checkpoint_4_{i}")

Logging to ./maskable_ppo_logs_4/PPO_5
Step 1000, 1s elapsed, 57s remaining
Step 2000, 2s elapsed, 41s remaining
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.33     |
|    ep_rew_mean     | -2.67    |
| time/              |          |
|    fps             | 1162     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
Step 3000, 4s elapsed, 67s remaining
Step 4000, 5s elapsed, 55s remaining
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 3.33          |
|    ep_rew_mean          | -2.67         |
| time/                   |               |
|    fps                  | 838           |
|    iterations           | 2             |
|    time_elapsed         | 4             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 1.164

KeyboardInterrupt: 

In [ ]:
model.save("mppo_model_150k_2025-06-18_1600")